# GUI INTERFACE FOR ADDRESS PARSING
***

> Aim of the program is to geocode the data in a csv file and check the results manually to determine the integrity of data obtained.if the output is found suitable ,options are added to save it. you can opt to move through the file and finally save all the data obtained into another csv file in a pre-determined format 

***

##### cookbook for pysimplegui
`https://github.com/PySimpleGUI/PySimpleGUI/blob/master/docs/cookbook.md`

***

##### Prerequisites
-  `pip install PySimpleGUI`
-  `pip install pandas`

***

##### instructions
1. run the program
1. Browse and select the csv file (note : csv file should only have a single column ie address and index)
1. click on the open button 
1. Select a range of data that you want to process
1. buttons  
    a.GeO button geocodes the address   
    b.Next & Previous button allow you to move through the data in both directions   
    c.Save button saves the geocoded data   
    d.Exit button saves all the data collected into a csv file   
1. rest you can figure out

***

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR028cXXPDLBEmmh4H6FSvlkT1u0rieUvN57LTqH-mlvfzkGcS01g" alt="Alt text" 
     title="Title text" height="100" width="100" 
         align="right"/><img src="https://wiki.openstreetmap.org/w/images/9/98/OSMF1.jpg" alt="Alt text" 
     title="Title text" height="100" width="100" 
         align="right"/>
         


In [ ]:
#for exit
import sys

#for dataentry
import pandas as pd

#for geocoding
import requests
import time
import json

#for python gui
import PySimpleGUI as sg 
sg.ChangeLookAndFeel('GreenTan')

###############################################################################################################################

# intial window to browse for csv file in your computer 
flag_filename=1
while flag_filename:
    #storing the response
    value = sg.Window('My Script').Layout([[sg.Text('Document to open')],      
                                            [sg.In(), sg.FileBrowse()],      
                                            [sg.CloseButton('Open'), sg.CloseButton('Cancel')]]).Read() 
    #check if cancel button is clicked
    if value[0]=='Cancel':
        print ("thank u for clicking cancel")
        sys.exit("hasta la vista")
    if not value[1][0]=='':    
        fname=value[1][0]
        flag_filename=0
        
# read the file using pandas
data=pd.read_csv(fname)
rows=data.count()[0]-1

###############################################################################################################################
#get info on slice of data that you wish to explore
layout=[[sg.Text('Indexes -> 0   -'),sg.Text(rows)],      
                                        [sg.Input(key='start_index'),sg.Input(key='stop_index')],      
                                            [sg.Button('next')]]
window = sg.Window('Index', layout)
event, values = window.Read()
window.Close()
if values['start_index'] == '':
    start=0
else:
    start=int(values['start_index'])
if values['stop_index'] == '':
    stop=rows
else:
    stop=int(values['stop_index'])

###############################################################################################################################
#create an intial dataframe to store your data    
c=['Raw_Address','display_name','Class','type','city','county','state_district','state','pincode']
df=pd.DataFrame(columns=c)

layout = [[sg.Text('\t\t\t\t\t\t\t\t\t\t\t')],
          #[sg.Button('Start')],
          [sg.Text('\n')],
          #sg.Input(do_not_clear=True, key='addrraw') 
          [sg.Text('Raw_Address '), sg.Multiline( key='addrraw',size=(35, 3)) , sg.Button('GeO') ],
          [sg.Frame(layout=[
              [sg.Text('Displayname', size=(15, 1)),sg.Input(key='display_name',do_not_clear=False)],
              [sg.Text('Class', size=(15, 1)),sg.Input(key='class',do_not_clear=False),sg.Text('Type', size=(15, 1)),sg.Input(key='type',do_not_clear=False)],
              [sg.Frame(layout=[
                  [sg.Text('City', size=(15, 1)),sg.Input(key='city',do_not_clear=False)],
                  [sg.Text('County', size=(15, 1)),sg.Input(key='county',do_not_clear=False)],
                  [sg.Text('State_district', size=(15, 1)),sg.Input(key='state_district',do_not_clear=False)],
                  [sg.Text('State', size=(15, 1)),sg.Input(key='state',do_not_clear=False)],
                  [sg.Text('Pincode', size=(15, 1)),sg.Input(key='pincode',do_not_clear=False)]
                        ], title='address',title_color='red', relief=sg.RELIEF_SUNKEN, tooltip='Use these to set flags')], 
              
            ], title='RESULTS',title_color='red', relief=sg.RELIEF_SUNKEN, tooltip='Use these to set flags')],
          [sg.Button('Save'),sg.Button('Previous'),sg.Button('Next'),sg.Text('\n\n')],
          [sg.Button('Exit')]] 

window = sg.Window('GeOcode!!!', layout) 
idisplayname=None
iclass=None
itype=None
icity=None
icounty=None
istate_district=None
istate=None
ipincode=None
text=''

index=start
# Event Loop 
while True:                 
    event, values = window.Read()  
    print(event, values) # to check for errors(optional)
#######################################################################  NEXT BUTTON
# next button gives the next available address in the csv file.if you finish checking data in the range you provide ,
# it saves the file automatically (default file format - "result_<starting index>-<stopping index>.csv")
    if event == 'Next':
        text=data.iloc[index,0]
        window.Element('addrraw').Update(data.iloc[index,0])
        if index>stop:
            name ="result_"+str(start)+"-"+str(stop)+".csv"
            name=str(name)
            df.to_csv(name)
            break
        index+=1
        
#######################################################################  PREVIOUS BUTTON 
# previous button takes you to the previous index data
    if event == 'Previous':
        if (index-1)>=0:
            index=index-1
            text=data.iloc[index,0]
            window.Element('addrraw').Update(data.iloc[index,0])
            
#######################################################################  SAVE BUTTON 
# save button appends the geocoded data into the dataframe
    if event =='Save':
        df=df.append({'Raw_Address':text,'display_name':idisplayname,'Class':iclass,'Type':itype,'city':icity,'county':icounty,'state_district':istate_district,'state':istate,'pincode':ipincode},ignore_index=True)    

#######################################################################  EXIT BUTTON
# exit button ends your gui session and save the data to the csv file
    if event is None or event == 'Exit':
        name ="result_"+str(start)+"-"+str(index-1)+".csv"
        name=str(name)
        df.to_csv(name)
        break
        
#######################################################################  GEO BUTTON 
# geo button is to geocode the data you entered and display the data in tabular form
    if event == 'GeO':
        text=values['addrraw']
        request = requests.get('https://nominatim.openstreetmap.org/?format=json&addressdetails=1&q='+text+'&format=json&countrycodes=in&bounded=1&viewbox=77.180737,12.732146,77.968935,13.504153')
        x=request.text
        print (request.status_code)
        
        if len(x)!= 2:
            parsed_x = json.loads(x)
            print (parsed_x[0])
            idisplayname=None
            iclass=None
            itype=None
            icity=None
            icounty=None
            istate_district=None
            istate=None
            ipincode=None
            if 'display_name' in parsed_x[0]:
                window.Element('display_name').Update(parsed_x[0]['display_name'])
                idisplayname=parsed_x[0]['display_name']
            if 'class' in parsed_x[0]:
                window.Element('class').Update(parsed_x[0]['class'])
                iclass=parsed_x[0]['class']
            if 'type' in parsed_x[0]:
                window.Element('type').Update(parsed_x[0]['type'])
                itype=parsed_x[0]['type']
            if 'city' in parsed_x[0]['address']:
                window.Element('city').Update(parsed_x[0]['address']['city'])
                icity=parsed_x[0]['address']['city']
            if 'county' in parsed_x[0]['address']:
                window.Element('county').Update(parsed_x[0]['address']['county'])
                icounty=parsed_x[0]['address']['county']
            if 'state_district' in parsed_x[0]['address']:
                window.Element('state_district').Update(parsed_x[0]['address']['state_district'])
                istate_district=parsed_x[0]['address']['state_district']
            if 'state' in parsed_x[0]['address']:
                window.Element('state').Update(parsed_x[0]['address']['state'])
                istate=parsed_x[0]['address']['state']
            if 'postcode' in parsed_x[0]['address']:
                window.Element('pincode').Update(parsed_x[0]['address']['postcode'])
                ipincode=parsed_x[0]['address']['postcode']

window.Close()

###############################################################################################################################
print ("Bye Bye")